# Peptide classification project

This notebooks shows preprocessing steps for Peptide group work, such as how we calculated the embeddings, and what selection/ quality control criteria were used to select the final subset of peptides for the project.

### Performed preprocessing steps:

In [1]:
import numpy as np
import pandas as pd
import esm
import torch

In [2]:
# full data table (see paper)
d = pd.read_excel("./data/TPDB/main.xlsx")
d.sample(5)

,ID,Function,Label encoding,Sequence,Source,Is_natural_peptide,HELM notation,N-terminal modification,C-terminal modification,Post translation modifications,Reference
20864,120865,Antimicrobial,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",IACIENKDTCRLKNCPRLHNVVGTCYEGKGKCCHK,Mus musculus,True,PEPTIDE1{[ac].I.A.C.I.E.N.K.D.T.C.R.L.K.N.C.P....,NaN,NaN,NaN,dramp(DRAMP04897)
47148,147149,Drug_delivery|Cell_penetrating,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",TAKTRYKARRAELIAERRGC,ND,True,PEPTIDE1{[ac].T.A.K.T.R.Y.K.A.R.R.A.E.L.I.A.E....,NaN,NaN,NaN,"J Biol Chem, 2001, 276, 5836;PMID: 11084031"
5123,105124,"Antimicrobial|Antibacterial|Anti-gram+,Antimic...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",CRWRLRLRC,ND,True,PEPTIDE1{[ac].C.R.W.R.L.R.L.R.C.[am]}$$$$,NaN,NaN,NaN,"Mar Drugs, 2021, 19, 451;PMID: 34436290"
47079,147080,Cell_Communication,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",SYSMEHFRWGKPVGR,animal,True,PEPTIDE1{[ac].S.Y.S.M.E.H.F.R.W.G.K.P.V.G.R.[a...,NaN,NaN,NaN,"Mol Cell Endocrinol, 1998, 143, 23;PMID: 9806347"
27340,127341,Antimicrobial,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",KVACCPSIAASNYYSICRLYGASGPKCAKIEDCKIVDGEECPGSTYP,Neurachne alopecuroidea|Lepidosperma gibsonii|...,True,PEPTIDE1{[ac].K.V.A.C.C.P.S.I.A.A.S.N.Y.Y.S.I....,NaN,NaN,NaN,"PLoS One, 2021, 16, e0254549;PMID: 34260649"


In [11]:
# FILTER RECORDS: Selecting a subset of the data for this exercise with 2 conditions
# 1: peptide length is between 20 and 50 (similar enough that the embeddings aren't biased by length)
# 2: The peptide consists of only 20 amino acids with capital letter string representation (no bugs in embedding step).
# -> This gives us 17.2k peptide sequences to work with 

d_sub = d.loc[(d["Sequence"].str.len() > 20) & (d["Sequence"].str.len() < 50) & (d["Sequence"].str.fullmatch('^[ACDEFGHIKLMNPQRSTVWY]*$')), :]
d_sub

,ID,Function,Label encoding,Sequence,Source,Is_natural_peptide,HELM notation,N-terminal modification,C-terminal modification,Post translation modifications,Reference
3,100004,"Antimicrobial|Antibacterial|Anti-gram+,Antimic...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,Human,True,PEPTIDE1{[ac].A.A.A.A.A.A.A.A.A.A.G.I.G.K.F.L....,NaN,NaN,NaN,"Antimicrob Agents Chemother, 1992, 36, 313;PMI..."
6,100007,"Antimicrobial|Antibacterial|Anti-gram+,Antimic...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",AAAAAAAIKMLMDLVNERIMALNKKAKK,virus (Bovine papular stomatitis virus)|Bovine...,True,PEPTIDE1{[ac].A.A.A.A.A.A.A.I.K.M.L.M.D.L.V.N....,NaN,NaN,NaN,"PLoS One, 2012, 7, e45848;PMID: 23029273"
12,100013,"Antimicrobial|Antibacterial|Anti-gram+,Antimic...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",AAAAGSCVWGAVNYTSDCAAECKRRGYKGGHCGSFANVNCWCET,ND,True,PEPTIDE1{[ac].A.A.A.A.G.S.C.V.W.G.A.V.N.Y.T.S....,NaN,NaN,NaN,"Biochemistry, 2001, 40, 11995;PMID: 11580275"
13,100014,"Antimicrobial|Antibacterial|Anti-gram+,Antimic...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",AAAAGSCVWGAVNYTSDCAAECKRRGYKGGHCGSFANVNCWCRT,ND,True,PEPTIDE1{[ac].A.A.A.A.G.S.C.V.W.G.A.V.N.Y.T.S....,NaN,NaN,NaN,"Biochemistry, 2001, 40, 11995;PMID: 11580275"
14,100015,"Antimicrobial|Antibacterial|Anti-gram+,Antimic...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",AAAAGSCVWGAVNYTSDCAAECLLRGYKGGHCGSFANVNCWCET,ND,True,PEPTIDE1{[ac].A.A.A.A.G.S.C.V.W.G.A.V.N.Y.T.S....,NaN,NaN,NaN,"Biochemistry, 2001, 40, 11995;PMID: 11580275"
...,...,...,...,...,...,...,...,...,...,...,...
54635,154636,"Antimicrobial|Antibacterial|Anti-gram+,Antimic...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",YYQVSEERRRDLASLARLYALAR,Pleurodema somuncurense,True,PEPTIDE1{[ac].Y.Y.Q.V.S.E.E.R.R.R.D.L.A.S.L.A....,NaN,NaN,NaN,"J Nat Prod, 2020, 83, 972;PMID: 32134261"
54645,154646,Neuropeptide,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",YYYGASPYAYSGGYYDSPYSY,fruit fly (Drosophila melanogaster),True,PEPTIDE1{[ac].Y.Y.Y.G.A.S.P.Y.A.Y.S.G.G.Y.Y.D....,NaN,NaN,NaN,Title: Peptidomics of the larval Drosophila me...
54647,154648,Antimicrobial,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",YYYKCFKDSDCVKLLCRIPLRPKCMYRHICKCKVVLTQNNYVLT,Synthetic construct,True,PEPTIDE1{[ac].Y.Y.Y.K.C.F.K.D.S.D.C.V.K.L.L.C....,NaN,NaN,NaN,dramp(DRAMP12482)
56927,156928,Antimicrobial|Antiviral|AntiHIV,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",GIGDPVTCLKSGAICHPVFCPRRYKQIGTCGLPGTKCCKKP,Homo sapiens,False,PEPTIDE1{[ac].G.I.G.D.P.V.T.C.L.K.S.G.A.I.C.H....,NaN,NaN,"Disulfide bonds between Cys8 and Cys37, Cys15 ...","J Virol, 2005, 79, 14318;PMID: 16254366"


In [12]:
# prepare for ESM-2 embedding format
sequences = list(zip(d_sub["ID"], d_sub["Sequence"]))
sequences[:5]

[(100004, 'AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS'),
 (100007, 'AAAAAAAIKMLMDLVNERIMALNKKAKK'),
 (100013, 'AAAAGSCVWGAVNYTSDCAAECKRRGYKGGHCGSFANVNCWCET'),
 (100014, 'AAAAGSCVWGAVNYTSDCAAECKRRGYKGGHCGSFANVNCWCRT'),
 (100015, 'AAAAGSCVWGAVNYTSDCAAECLLRGYKGGHCGSFANVNCWCET')]

In [8]:
# Load ESM-2 model
esm_model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
esm_model.eval()  # disables dropout for deterministic results

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((1280,), eps=1

In [9]:
def create_esm_embeddings(records, batch_size = 10):
    "Given a list of (name, sequence) tuples, return 1280 dimensional embeddings for the sequences using esm 2 (in minibatches)"
    
    print(f"Starting: 0 / {len(records)} processed")
    res = np.zeros([len(records), 1280]) # all embeddings
    sequence_representations = [] # batch embeddings
    
    for idx in range(0, len(records), batch_size):
        sub_r = records[idx: idx+batch_size]
        batch_labels, batch_strs, batch_tokens = batch_converter(sub_r)
        batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
    
    # Extract per-residue representations (on CPU)
        #print("starting embedding")
        with torch.no_grad():
            results = esm_model(batch_tokens, repr_layers=[33], return_contacts=False)
        # return token (amino acid) level representations
        token_representations = results["representations"][33]
        #print("Ended embedding")
    
        # summarise sequence embeddings by taking an average representation of individual amino acis representations
        #sequence_representations = []
        for i, tokens_len in enumerate(batch_lens):
            sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0).numpy()) # 1:st token = beginning of sequence
        
        res[idx:(idx+batch_size), :] = np.array(sequence_representations)
        sequence_representations = []
        
        if idx % 100 == 0:
            print(f"{idx + batch_size}/{len(records)} processed")
    return res

In [ ]:
embeddings = create_esm_embeddings(sequences)
embeddings

In [123]:
# Expanding the label Encoding table as it's own table for convenience (order from main.xls)
labels = ['Analgesic', 'Angiogenic', 'Anti-gram+', 'Anti-gram-', 'AntiAngiogenesis', 'AntiBreastcancer', 'AntiCervicalcancer', 
          'AntiColoncancer', 'AntiHCV', 'AntiHIV', 'AntiHSV', 'AntiLivercancer', 'AntiLungcancer', 'AntiMERS-Cov', 'AntiProstatecancer', 
          'AntiSARS', 'AntiSkincancer', 'AntiTubercular', 'Antibacterial', 'Anticancer', 'Antifungal', 'Antihypertensive', 
          'Antiinflammatory', 'Antileishmania', 'Antimalarial', 'Antimicrobial', 'Antioxidant', 'Antiparasitic', 'Antiplasmodial', 
          'Antitrypanosomic', 'Antiviral', 'Blood-brain_barrier_penetrating', 'Cell_Communication','Cell_penetrating', 'Drug_delivery', 
          'Glucose_metabolism', 'Growth_Regeneration', 'Growth_regeneration', 'Growth_regulatory', 'Immunoactive', 'Lipid_metabolism', 
          'Metabolic_regulatory', 'Neuropeptide', 'Opioid', 'Osteogenic', 'Quorum_sensing', 'Thrombolytic', 'Tumor_homing']


labels_wide = d_sub["Label encoding"].str.slice(1, -1).str.split(". ", expand=True).astype("int")
labels_wide.columns = labels
labels_wide

,Analgesic,Angiogenic,Anti-gram+,Anti-gram-,AntiAngiogenesis,AntiBreastcancer,AntiCervicalcancer,AntiColoncancer,AntiHCV,AntiHIV,...,Growth_regulatory,Immunoactive,Lipid_metabolism,Metabolic_regulatory,Neuropeptide,Opioid,Osteogenic,Quorum_sensing,Thrombolytic,Tumor_homing
ID,,,,,,,,,,,,,,,,,,,,,
100004,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100007,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100013,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100014,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100015,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154636,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
154646,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
154648,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
# For this exercise, let's create 2 helper columns:
# 1: isSinglepurpose -> Boolean that show that peptide belongs to just 1 main category
# 2: Category: name of the main category if it exists, otherwise "Multipurpose"
# main classes:
main_categories = labels_wide.loc[:, ["Immunoactive", "Anticancer", "Antihypertensive", "Neuropeptide", "Antioxidant", "Metabolic_regulatory",
                   "Cell_Communication", "Thrombolytic", "Antibacterial", "Antifungal", "Antiparasitic", "Antiviral",
                   "Antimicrobial", "Drug_delivery", "Growth_regeneration"]]


In [125]:
main_categories

,Immunoactive,Anticancer,Antihypertensive,Neuropeptide,Antioxidant,Metabolic_regulatory,Cell_Communication,Thrombolytic,Antibacterial,Antifungal,Antiparasitic,Antiviral,Antimicrobial,Drug_delivery,Growth_regeneration
ID,,,,,,,,,,,,,,,
100004,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
100007,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
100013,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
100014,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
100015,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154636,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
154646,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
154648,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [144]:
main_categories.loc[main_categories.sum(axis = 1) > 1, :] = 0
main_class = pd.from_dummies(main_categories, default_category="Multipurpose", )
main_class.columns = ["MAIN_Category"]
main_class["isSinglepurpose"] = main_class["MAIN_Category"] != "Multipurpose"
main_class

,MAIN_Category,isSinglepurpose
ID,,
100004,Antibacterial,True
100007,Antibacterial,True
100013,Multipurpose,False
100014,Multipurpose,False
100015,Multipurpose,False
...,...,...
154636,Antibacterial,True
154646,Neuropeptide,True
154648,Antimicrobial,True


In [146]:
Y = pd.concat([main_class, labels_wide], axis = 1)
Y

,MAIN_Category,isSinglepurpose,Analgesic,Angiogenic,Anti-gram+,Anti-gram-,AntiAngiogenesis,AntiBreastcancer,AntiCervicalcancer,AntiColoncancer,...,Growth_regulatory,Immunoactive,Lipid_metabolism,Metabolic_regulatory,Neuropeptide,Opioid,Osteogenic,Quorum_sensing,Thrombolytic,Tumor_homing
ID,,,,,,,,,,,,,,,,,,,,,
100004,Antibacterial,True,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100007,Antibacterial,True,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100013,Multipurpose,False,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100014,Multipurpose,False,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100015,Multipurpose,False,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154636,Antibacterial,True,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
154646,Neuropeptide,True,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
154648,Antimicrobial,True,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# save derived tables (uncomment to save):
# embedding_df = pd.DataFrame(embeddings, index = d_sub["ID"])
# embedding_df.to_csv("./data/Embeddings_Esm2.csv")

# Y.to_csv("./data/Peptide_Targetlabels.csv")
# d_sub.set_index("ID").to_csv("./data/Peptide_Metadata.csv")